In [1]:
from product import Option
from strategy import *
from euronext import Page
from graph import Graph
g=Graph(4800, 5500, 25)

In [2]:
p = Page()
p.fetch()
options = p.scrap_options()
call = p.call
put = p.put
for k, o in p.call.items():
    o.multiplier=10
for k, o in p.put.items():
    o.multiplier=10


In [4]:
g.display_page_raw(p)

Settl.C,OIC,Day Vol C,Last C,bid C,ask C,C,strike,P,bid P,ask P,Last P,Day Vol P,OIP,Settl.P
1101.55,-,-,-,1097.70,1106.1,C,4000,P,-,0.4,-,-,568,0.01
901.55,-,-,-,897.70,906.7,C,4200,P,-,0.5,0.10,4,489,0.01
701.62,-,-,-,697.80,706.3,C,4400,P,-,0.5,-,-,"1,069",0.1
601.7,-,-,-,597.80,606.9,C,4500,P,0.10,0.6,-,-,719,0.18
551.75,-,-,-,548.40,556.9,C,4550,P,0.10,0.7,-,-,63,0.23
501.8,-,-,-,497.90,506.5,C,4600,P,0.10,0.7,0.70,15,"1,005",0.28
451.87,-,-,-,448.40,457.1,C,4650,P,0.20,0.8,-,-,70,0.35
401.96,-,-,-,398.00,407.2,C,4700,P,0.30,0.9,-,-,927,0.44
352.22,-,-,-,348.60,357.3,C,4750,P,0.30,1.1,1.10,20,117,0.71
302.27,9,-,-,298.30,307,C,4800,P,0.40,0.9,1.20,1,758,0.76


In [ ]:
169.9-(2*77.30)+19.7

In [ ]:
call[4900].vente

In [ ]:
# test data when interet is fall
import numpy as np
sT = np.arange(4800,5400,25)
o = Option(strike=4900, achat=212.40, vente=219.40, multiplier=10)
c = Option(cat='Call', strike=4900, achat=212.40, vente=219.40, multiplier=10)
p = Option(cat='Put', strike=4900, achat=9.10, vente=11.60, multiplier=10)
print(str(o));print(str(c));print(str(p))

In [ ]:
s2 = Strategy('Strategy : call+2*put long').add(call[4900], 'long', 1).add(call[5300], 'short', 2)
g.profit(call[4950], 'long')
g.profit_strategy(s2)
g.compare(call[5000], 'long', s2, 'long')

In [ ]:
butterfly = (Strategy('Butterfly C5000-2*C5100+C5200')
             .add(call[4900], 'long', 1)
             .add(call[5000], 'short', 2)
             .add(call[5100], 'long', 1))
call_strike = 5000
g.profit_strategy(butterfly)
g.compare(call[call_strike], 'long', butterfly, 'long')
print('Cost-Butterfly : {:0.02f}€'.format(butterfly.cost()))
print('Cost-Call : {:0.02f}€'.format(call[call_strike].cost('long')))
g.display_summary(butterfly)

In [ ]:
ratio_backspreads = (Strategy('Ratio Backspreads')
                     .add(call[4900], 'long', 1)
                     .add(call[5100], 'short', 3)
                    )
g.profit_strategy(ratio_backspreads)
g.compare(ratio_backspreads, 'long', call[4900], 'long')
print('Ratio Backspreads Cost : {:0.02f}€'.format(ratio_backspreads.cost()))
print('Call 5000 Cost : {:0.02f}€'.format(call[4900].cost('long')))
g.display_summary(ratio_backspreads)

In [ ]:
straddle = (Strategy('Straddle C5100+P5100')
             .add(call[4950], 'short', 1)
             .add(put[5150], 'short', 1))
print('Straddle Cost : {}'.format(straddle.cost()))
g.profit_strategy(straddle)

In [ ]:

payoff_max=max(straddle.payoff(g.sT, 'long'))
payoff_min=min(straddle.payoff(g.sT, 'long'))
{'min': payoff_min, 'max': payoff_max, 'cost': g.profit_strategy(straddle)}

In [ ]:
straddle = (Strategy('Straddle C5100+P5100')
             .add(call[4950], 'short', 1)
             .add(put[5150], 'short', 1))
print('Straddle Cost : {}'.format(straddle.cost()))
g.profit_strategy(straddle)
c = put[4950]
print('Call Cost : {}'.format(c.cost('short')))
g.profit(c, 'short')

In [ ]:
for s, c in call.items():
    print('{} {} {}'.format(c.strike, c.multiplier, c.vente))

In [ ]:
strat1 = (Strategy('Stratis 5100-..-5300 croissant')
             .add(put[5000], 'short', 3)
             .add(put[5100], 'short', 2)
             .add(put[5200], 'short', 1)
        )
strat2 = (Strategy('Stratis 5100-..-5300 decroissant')
             .add(put[5000], 'short', 1)
             .add(put[5100], 'short', 2)
             .add(put[5200], 'short', 3)
        )
print('{} Cost : {}'.format(strat1.label, strat1.cost()))
print('{} Cost : {}'.format(strat2.label, strat2.cost()))
g.profit_strategy(strat1)
g.profit_strategy(strat2)
g.compare(strat1, 'long', strat2, 'long')

In [ ]:
g.profit(call[4950], 'long')
print('cost: {}'.format(call[4950].cost('long')))
g.profit(call[4900], 'long')
print('cost: {}'.format(call[4900].cost('long')))


In [ ]:
#ratio-spreads
rspread2 = RatioSpread(call[4950], call[5150])
print('{} Cost : {}'.format(rspread2.label, rspread2.cost()))
g.profit_strategy(rspread2)

In [ ]:
#????
#box-spreads pour John Hull
from math import ceil, floor
def BoxSpread(call_long, call_short, put_long, put_short):
    """A Box Spread is buy and sell put and call of different strike:
       - short a put at a strike
       - long a put at a strike inferior
       This a strategy of hedgin not speculatif
    """
    plong = call_long.achat
    pshort = call_short.vente
    plong = put_long.achat
    pshort = put_short.vente
    return (Strategy('Box Spread {}-{}'.format(put_long.strike, put_short.strike))
            .add(call_long, 'short', 1)
            .add(call_short, 'long', 1)
            .add(put_long, 'short', 1)
            .add(put_short, 'long', 1)
           )

In [ ]:
bspread2 = BoxSpread(call[5000], call[4900], put[5100], put[5200])
print('{} Cost : {}'.format(bspread2.label, bspread2.cost()))
bspread1 = BoxSpread(call[4950], call[4900], put[5150], put[5200])
print('{} Cost : {}'.format(bspread1.label, bspread1.cost()))
g.profit_strategy(bspread2)
g.profit_strategy(bspread1)
g.compare(bspread2, 'long', bspread1, 'long')
g.display_summary(bspread2)

In [ ]:
cspread1 = CallSpread(call[4900], call[5000])
print('{} Cost : {}'.format(cspread1.label, cspread1.cost()))
cspread2 = CallSpread(call[4900], call[5150])
print('{} Cost : {}'.format(cspread2.label, cspread2.cost()))
g.profit_strategy(cspread1)
g.profit_strategy(cspread2)
g.compare(cspread1, 'long', cspread2, 'long')

In [ ]:
pspread1 = PutSpread(put[5200], put[5000])
print('{} Cost : {}'.format(pspread1.label, pspread1.cost()))
g.profit_strategy(pspread1)
pspread2 = PutSpread(put[5100], put[5000])
print('{} Cost : {}'.format(pspread2.label, pspread2.cost()))
g.profit_strategy(pspread2)
g.compare(pspread1, 'long', pspread2, 'long')

In [ ]:
ironcondor = IronCondor(put[5050], put[5000], call[5250], call[5300])
print('{} Cost : {}'.format(ironcondor.label, ironcondor.cost()))
risk = min(ironcondor.payoff(g.sT, 'long'))
print('{} Risk : {}'.format(ironcondor.label,risk))
g.profit_strategy(ironcondor)
g.display_summary(ironcondor)

In [ ]:
help(IronCondor)